In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
# Ignore the warnings
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')

In [3]:
# Importing various modules for data visualisation and manipulation
import numpy as np
import pandas as pd 
from csv import reader
from pandas import Series, DataFrame
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt 
from matplotlib import style
import seaborn as sns
import glob
import collections
import networkx as nx
import gc
from collections import Counter
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [4]:
# Preamble 
# import sherlockml.filesystem as sfs
import random
from tkinter import _flatten
from xgboost import XGBClassifier
from sklearn.preprocessing import StandardScaler 

In [5]:
df = pd.read_csv('./df_3.csv')
del df['Unnamed: 0']
# df = df.set_index('id')
df

,id,user_id,age_range,gender,merchant_id,label,frequency,monetary,recency,repur_mers,...,umi_max,umi_min,umb_size,umb_mean,umb_max,umb_min,umc_size,umc_mean,umc_max,umc_min
0,34176_3906,34176,6.0,0.0,3906,0,451,34,4,1.0,...,16,1,1,39.0,39,39,6,6.5,24,1
1,34176_121,34176,6.0,0.0,121,0,451,34,4,1.0,...,14,14,1,14.0,14,14,1,14.0,14,14
2,34176_4356,34176,6.0,0.0,4356,1,451,34,4,1.0,...,17,1,1,18.0,18,18,1,18.0,18,18
3,34176_2217,34176,6.0,0.0,2217,0,451,34,4,1.0,...,2,2,1,2.0,2,2,1,2.0,2,2
4,230784_4818,230784,0.0,0.0,4818,0,54,7,27,0.0,...,8,8,1,8.0,8,8,1,8.0,8,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
260859,359807_4325,359807,4.0,1.0,4325,0,117,9,8,0.0,...,14,1,1,20.0,20,20,2,10.0,15,5
260860,294527_3971,294527,0.0,1.0,3971,0,198,5,47,0.0,...,9,4,1,17.0,17,17,1,17.0,17,17
260861,294527_152,294527,0.0,1.0,152,0,198,5,47,0.0,...,9,9,1,9.0,9,9,1,9.0,9,9
260862,294527_2537,294527,0.0,1.0,2537,0,198,5,47,0.0,...,1,1,1,1.0,1,1,1,1.0,1,1


In [6]:
del_col = [i for i in df.columns if len(set(df[i].values)) == 1]
df.drop(columns = del_col, axis = 1, inplace = True)
df.shape

(260864, 157)

## Data Transform

### categorical columns

In [7]:
cat_col = ['age_range', 'gender']
prefix_col = ['age','gender']
cat_col

['age_range', 'gender']

In [8]:
df1= df.copy()
for i in range(len(cat_col)):
    df1 = df1.join(pd.get_dummies(df[cat_col[i]], prefix = prefix_col[i],drop_first = True))
df1

,id,user_id,age_range,gender,merchant_id,label,frequency,monetary,recency,repur_mers,...,age_1.0,age_2.0,age_3.0,age_4.0,age_5.0,age_6.0,age_7.0,age_8.0,gender_1.0,gender_2.0
0,34176_3906,34176,6.0,0.0,3906,0,451,34,4,1.0,...,0,0,0,0,0,1,0,0,0,0
1,34176_121,34176,6.0,0.0,121,0,451,34,4,1.0,...,0,0,0,0,0,1,0,0,0,0
2,34176_4356,34176,6.0,0.0,4356,1,451,34,4,1.0,...,0,0,0,0,0,1,0,0,0,0
3,34176_2217,34176,6.0,0.0,2217,0,451,34,4,1.0,...,0,0,0,0,0,1,0,0,0,0
4,230784_4818,230784,0.0,0.0,4818,0,54,7,27,0.0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
260859,359807_4325,359807,4.0,1.0,4325,0,117,9,8,0.0,...,0,0,0,1,0,0,0,0,1,0
260860,294527_3971,294527,0.0,1.0,3971,0,198,5,47,0.0,...,0,0,0,0,0,0,0,0,1,0
260861,294527_152,294527,0.0,1.0,152,0,198,5,47,0.0,...,0,0,0,0,0,0,0,0,1,0
260862,294527_2537,294527,0.0,1.0,2537,0,198,5,47,0.0,...,0,0,0,0,0,0,0,0,1,0


## VIF Selection
### Multicollneairty

In [9]:
def vif(df,threshold):
    multi_list = [i for i in df.columns if i not in ['user_id','merchant_id','id','label']]
    vif_list = [variance_inflation_factor(df[multi_list].values, i) for i in range(df[multi_list].shape[1])]
    vif_df = pd.DataFrame([multi_list,vif_list]).T
    vif_df.columns = ['name','vif']
    sel_df = vif_df[vif_df['vif'] < threshold]
    # threshold = sqrt(feature numbers)
    sel_df_1 = df[['user_id','merchant_id','id','label']+[i for i in df.columns if i in sel_df.name.values]]
    return sel_df_1

In [10]:
vif_df  = vif(df1,15)
vif_df

,user_id,merchant_id,id,label,recency,repur_mers,cart_r,buy_r,ncart,nbuy,...,i_min,c_size,c_mean,c_min,b_size,m_cart_r_y,time_range_y,ts_count,umi_min,umc_size
0,34176,3906,34176_3906,0,4,1.0,0.0,0.075388,0.0,14.0,...,1,20,813.450000,5,1,0.0,21,9,1,6
1,34176,121,34176_121,0,4,1.0,0.0,0.075388,0.0,14.0,...,1,26,3071.730769,1,1,0.0,3,3,14,1
2,34176,4356,34176_4356,1,4,1.0,0.0,0.075388,0.0,14.0,...,1,15,484.600000,1,1,0.0,1,2,1,1
3,34176,2217,34176_2217,0,4,1.0,0.0,0.075388,0.0,14.0,...,1,5,12040.400000,614,1,0.0,0,1,2,1
4,230784,4818,230784_4818,0,27,0.0,0.0,0.129630,0.0,5.0,...,1,27,1781.074074,2,1,0.0,3,3,8,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
260859,359807,4325,359807_4325,0,8,0.0,0.0,0.076923,0.0,2.0,...,1,18,1099.388889,7,1,0.0,0,1,1,2
260860,294527,3971,294527_3971,0,47,0.0,0.0,0.025253,0.0,1.0,...,1,7,4127.428571,4,1,0.0,2,2,4,1
260861,294527,152,294527_152,0,47,0.0,0.0,0.025253,0.0,1.0,...,1,13,623.615385,1,1,0.0,0,1,9,1
260862,294527,2537,294527_2537,0,47,0.0,0.0,0.025253,0.0,1.0,...,1,21,2290.190476,1,1,0.0,0,1,1,1


### Correlation

In [11]:
features = [x for x in vif_df.columns]
abs_corr = vif_df[features].corr().abs()
corr_series = abs_corr.unstack()
corr = sorted(corr_series.items(),key = lambda x: x[1], reverse=True) 
corr = [corr[x] for x in range(len(corr)) if corr[x][1]!=1]
corr

[(('ub_max', 'uc_max'), 0.8246182555927477),
 (('uc_max', 'ub_max'), 0.8246182555927477),
 (('label', 'repur_mers'), 0.8108745745509044),
 (('repur_mers', 'label'), 0.8108745745509044),
 (('nbuy_r', 'ndays_ratio'), 0.7542895198757839),
 (('ndays_ratio', 'nbuy_r'), 0.7542895198757839),
 (('ui_max', 'uc_max'), 0.718512378488889),
 (('uc_max', 'ui_max'), 0.718512378488889),
 (('ui_max', 'ub_max'), 0.6664314249949678),
 (('ub_max', 'ui_max'), 0.6664314249949678),
 (('ub_mean', 'ub_max'), 0.6612503445624159),
 (('ub_max', 'ub_mean'), 0.6612503445624159),
 (('c_size', 'b_size'), 0.6493858084430958),
 (('b_size', 'c_size'), 0.6493858084430958),
 (('ui_mean', 'ui_max'), 0.6087870521218965),
 (('ui_max', 'ui_mean'), 0.6087870521218965),
 (('buy_r', 'nbuy_r'), 0.5921065149669631),
 (('nbuy_r', 'buy_r'), 0.5921065149669631),
 (('time_range_y', 'ts_count'), 0.5898196492819222),
 (('ts_count', 'time_range_y'), 0.5898196492819222),
 (('ui_mean', 'ub_mean'), 0.5863533562051397),
 (('ub_mean', 'ui_mea

In [12]:
high_co_pair = [corr[i][0] for i in range(len(corr)) if corr[i][1] > 0.9]
high_co_pair

[]

In [13]:
# Done

### Standard Scale

In [14]:
vif_df = vif_df.set_index('id')
vif_df

,user_id,merchant_id,label,recency,repur_mers,cart_r,buy_r,ncart,nbuy,nfav,...,i_min,c_size,c_mean,c_min,b_size,m_cart_r_y,time_range_y,ts_count,umi_min,umc_size
id,,,,,,,,,,,,,,,,,,,,,
34176_3906,34176,3906,0,4,1.0,0.0,0.075388,0.0,14.0,0.0,...,1,20,813.450000,5,1,0.0,21,9,1,6
34176_121,34176,121,0,4,1.0,0.0,0.075388,0.0,14.0,0.0,...,1,26,3071.730769,1,1,0.0,3,3,14,1
34176_4356,34176,4356,1,4,1.0,0.0,0.075388,0.0,14.0,0.0,...,1,15,484.600000,1,1,0.0,1,2,1,1
34176_2217,34176,2217,0,4,1.0,0.0,0.075388,0.0,14.0,0.0,...,1,5,12040.400000,614,1,0.0,0,1,2,1
230784_4818,230784,4818,0,27,0.0,0.0,0.129630,0.0,5.0,0.0,...,1,27,1781.074074,2,1,0.0,3,3,8,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
359807_4325,359807,4325,0,8,0.0,0.0,0.076923,0.0,2.0,0.0,...,1,18,1099.388889,7,1,0.0,0,1,1,2
294527_3971,294527,3971,0,47,0.0,0.0,0.025253,0.0,1.0,1.0,...,1,7,4127.428571,4,1,0.0,2,2,4,1
294527_152,294527,152,0,47,0.0,0.0,0.025253,0.0,1.0,1.0,...,1,13,623.615385,1,1,0.0,0,1,9,1


In [15]:
scaler = StandardScaler()
scaled_df = scaler.fit_transform(vif_df.drop(['user_id','merchant_id','label'], axis=1))

In [16]:
vif_df.drop(['user_id','merchant_id','label'], axis=1)

,recency,repur_mers,cart_r,buy_r,ncart,nbuy,nfav,ncart_r,nbuy_r,ndays_ratio,...,i_min,c_size,c_mean,c_min,b_size,m_cart_r_y,time_range_y,ts_count,umi_min,umc_size
id,,,,,,,,,,,,,,,,,,,,,
34176_3906,4,1.0,0.0,0.075388,0.0,14.0,0.0,0.0,0.072917,0.423077,...,1,20,813.450000,5,1,0.0,21,9,1,6
34176_121,4,1.0,0.0,0.075388,0.0,14.0,0.0,0.0,0.072917,0.423077,...,1,26,3071.730769,1,1,0.0,3,3,14,1
34176_4356,4,1.0,0.0,0.075388,0.0,14.0,0.0,0.0,0.072917,0.423077,...,1,15,484.600000,1,1,0.0,1,2,1,1
34176_2217,4,1.0,0.0,0.075388,0.0,14.0,0.0,0.0,0.072917,0.423077,...,1,5,12040.400000,614,1,0.0,0,1,2,1
230784_4818,27,0.0,0.0,0.129630,0.0,5.0,0.0,0.0,0.135135,0.363636,...,1,27,1781.074074,2,1,0.0,3,3,8,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
359807_4325,8,0.0,0.0,0.076923,0.0,2.0,0.0,0.0,0.086957,0.400000,...,1,18,1099.388889,7,1,0.0,0,1,1,2
294527_3971,47,0.0,0.0,0.025253,0.0,1.0,1.0,0.0,0.050000,1.000000,...,1,7,4127.428571,4,1,0.0,2,2,4,1
294527_152,47,0.0,0.0,0.025253,0.0,1.0,1.0,0.0,0.050000,1.000000,...,1,13,623.615385,1,1,0.0,0,1,9,1


In [17]:
vif_df

,user_id,merchant_id,label,recency,repur_mers,cart_r,buy_r,ncart,nbuy,nfav,...,i_min,c_size,c_mean,c_min,b_size,m_cart_r_y,time_range_y,ts_count,umi_min,umc_size
id,,,,,,,,,,,,,,,,,,,,,
34176_3906,34176,3906,0,4,1.0,0.0,0.075388,0.0,14.0,0.0,...,1,20,813.450000,5,1,0.0,21,9,1,6
34176_121,34176,121,0,4,1.0,0.0,0.075388,0.0,14.0,0.0,...,1,26,3071.730769,1,1,0.0,3,3,14,1
34176_4356,34176,4356,1,4,1.0,0.0,0.075388,0.0,14.0,0.0,...,1,15,484.600000,1,1,0.0,1,2,1,1
34176_2217,34176,2217,0,4,1.0,0.0,0.075388,0.0,14.0,0.0,...,1,5,12040.400000,614,1,0.0,0,1,2,1
230784_4818,230784,4818,0,27,0.0,0.0,0.129630,0.0,5.0,0.0,...,1,27,1781.074074,2,1,0.0,3,3,8,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
359807_4325,359807,4325,0,8,0.0,0.0,0.076923,0.0,2.0,0.0,...,1,18,1099.388889,7,1,0.0,0,1,1,2
294527_3971,294527,3971,0,47,0.0,0.0,0.025253,0.0,1.0,1.0,...,1,7,4127.428571,4,1,0.0,2,2,4,1
294527_152,294527,152,0,47,0.0,0.0,0.025253,0.0,1.0,1.0,...,1,13,623.615385,1,1,0.0,0,1,9,1


In [18]:
vif_df.to_csv('vif.csv')